In [ ]:
import pandas as pd
import numpy as np

file = "../data/facebook.csv"

df = pd.read_csv(file, low_memory=False, parse_dates=[0])

In [ ]:
%cd ..

In [ ]:
from src.clean_data import clean_df

In [ ]:
new_df = clean_df(df)
new_df.index = pd.to_datetime(new_df.index)
new_df.head()

In [ ]:
impressions = []
views = []
clicks = []
others = []

for col in new_df.columns:
    if "Impressions" in col or "Reach" in col:
        impressions.append(col)
    elif "Clicks" in col:
        clicks.append(col)
    elif "Views" in col:
        views.append(col)
    else:
        others.append(col)

## Impressions and Reach

In [ ]:
valid_impre = new_df[(new_df["Impressions"] != 0) & ~new_df["Impressions"].isna()]

con_df = (
    valid_impre.groupby("Content Type")["Impressions"]
    .mean()
    .reset_index()
    .sort_values("Impressions")
)

In [ ]:
from bokeh.io import output_notebook, export_png

output_notebook()

In [ ]:
from bokeh.plotting import figure, show

p = figure(
    title="Average Impression by Content Type",
    x_range=con_df["Content Type"],
    height=400,
    width=600,
    y_axis_label="Average Impression",
)

p.vbar(x=con_df["Content Type"], top=con_df["Impressions"], width=0.8)

p.y_range.start = 1000

show(p)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-whitegrid")

In [ ]:
mmi = valid_impre["Impressions"].resample("M").mean()
ax = mmi.fillna(method="ffill").plot(title="Average Impressions per month")

plt.savefig("docs/avg_month.png")
plt.show();

In [ ]:
cbt = valid_impre.groupby(["Content Type", "Time Period"])["Reach"].mean().reset_index()

time_period_order = ["morning", "afternoon", "evening"]

cbt["Time Period"] = pd.Categorical(
    cbt["Time Period"], categories=time_period_order, ordered=True
)
cbt = cbt.sort_values(["Content Type", "Time Period"])
cbt.reset_index(drop=True, inplace=True)

In [ ]:
link = cbt.iloc[:3]
link.name = "Link"

photo = cbt.iloc[3:6]
photo.name = "Photo"

text = cbt.iloc[6:9]
text.name = "Text"

video = cbt.iloc[9:]
video.name = "Video"

content = (link, photo, text, video)

In [ ]:
def plot_bar(df):
    p = figure(title=df.name, x_range=list(df["Time Period"]), height=300, width=400)

    p.vbar(x="Time Period", top="Reach", color="#99CCFF", source=df, width=0.8)

    p.xgrid.grid_line_color = None
    p.xaxis.major_tick_out = 0
    p.xaxis.axis_label = "Time period"
    p.yaxis.axis_label = "Average Reach"
    p.yaxis.minor_tick_out = 0
    p.yaxis.major_tick_out = 0
    p.y_range.start = 1000
    p.y_range.end = 9000

    return p

In [ ]:
from bokeh.layouts import gridplot

plots = []
for c in content:
    plot = plot_bar(c)
    plots.append(plot)

layout = gridplot(plots, ncols=2, toolbar_location=None)
show(layout)